In [27]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import numpy as np
import nltk
import pandas as pd
from ast import literal_eval
from collections import Counter
from scipy.optimize import fmin_l_bfgs_b
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
import multiprocessing
import random
from tqdm import tqdm
import optimizeTopicVectors as ot
import time

In [4]:
# hoseong's class
from STMD import *
from ASUM import *
from ASUM_Embedding import *
from preprocess import *

In [12]:
with open('D:/Dropbox/2016-2/졸업논문/STMD/data/SentiWords-0.txt','r') as f:
    pos = f.readlines()
pos_seed = [word.strip() for word in pos]
with open('D:/Dropbox/2016-2/졸업논문/STMD/data/SentiWords-1.txt','r') as f:
    neg = f.readlines()
neg_seed = [word.strip() for word in neg]

In [5]:
# load data
work_path = "/media/hs-ubuntu/data/dataset/MasterThesis/STMD_data/"
# work_path = "E:/dataset/MasterThesis/STMD_data/"

data = pd.read_csv(work_path + "preprocess_complete_Electronics.csv")

In [6]:
data.head()

,reviewTime,asin,reviewerID,overall,helpful,reviewText,title,brand,reviewSentence,sent_length,reviewSentence_tagged,preprocessed
0,2013-07-21,B00CM0XHNS,A372YX80GGM7DR,5.0,576,"Ok, so I didn't buy this on Amazon, as I didn'...",Ultimate Ears BOOM Wireless Bluetooth Speaker ...,Logitech,"[""Ok, so I didn't buy this on Amazon, as I did...",58,"[[('Ok', 'NNP'), (',', ','), ('so', 'IN'), ('I...","[['ok', 'not_buy', 'amazon', 'not_want', 'wait..."
1,2013-05-19,B00BQ5RY1G,A1BG2Z071TYO7P,2.0,522,I received a Harmony Ultimate from Logitech be...,Logitech Harmony Ultimate Remote with Customiz...,Logitech,['I received a Harmony Ultimate from Logitech ...,27,"[[('I', 'PRP'), ('received', 'VBD'), ('a', 'DT...","[['receiv', 'harmoni', 'ultim', 'logitech', 's..."
2,2013-12-16,B00EZ9XG62,AELAESM03451,1.0,290,This review is for the iPad Air keyboard. I ha...,Logitech Ultrathin Keyboard Cover for iPad Air...,Logitech,"['This review is for the iPad Air keyboard.', ...",23,"[[('This', 'DT'), ('review', 'NN'), ('is', 'VB...","[['review', 'ipad', 'air', 'keyboard'], ['keyb..."
3,2013-01-21,B0099SMFVQ,A36CMGR5ELUM34,5.0,283,Design: Very well put together. Elegant and th...,Logitech Bluetooth Illuminated Keyboard K810 f...,Logitech,"['Design: Very well put together.', 'Elegant a...",28,"[[('Design', 'NN'), (':', ':'), ('Very', 'RB')...","[['design', 'veri', 'well', 'put', 'togeth'], ..."
4,2013-07-29,B00CM0XHNS,A9TETE58A7JR3,3.0,260,"So, I've been testing a few bluetooth speakers...",Ultimate Ears BOOM Wireless Bluetooth Speaker ...,Logitech,"[""So, I've been testing a few bluetooth speake...",57,"[[('So', 'RB'), (',', ','), ('I', 'PRP'), (""'v...","[['test', 'bluetooth', 'speaker', 'week'], ['a..."


In [7]:
def extract_sample(data, brand_name, count, ratio = 1, random_state = 42):
    brand = data[data['brand']==brand_name]
    pos_reviews = brand[brand.overall >= 4]
    neg_reviews = brand[brand.overall <= 2]
    if ratio == 1:
        pos_sample = pos_reviews.sample(count, random_state=random_state)
        neg_sample = neg_reviews.sample(count, random_state=random_state)
        df = pd.concat([pos_sample, neg_sample], axis=0)
    else:
        df = brand.sample(count * 2, random_state = random_state)
    df['preprocessed'] = df.preprocessed.apply(lambda row: literal_eval(row))
    return df

In [37]:
# brand = ['Apple', 'Samsung','Canon']
# brand_df = data[data.brand.isin(brand)]
# brand_df.reset_index(drop=True, inplace=True)
# #긍정, 부정 반반씩
# pos_reviews = brand_df[brand_df.overall >= 4]
# neg_reviews = brand_df[brand_df.overall <= 2]
# pos_sample = pos_reviews.sample(3500, random_state=23)
# neg_sample = neg_reviews.sample(3500, random_state=42)
# df = pd.concat([pos_sample, neg_sample], axis=0)
# df['preprocessed'] = df.preprocessed.apply(lambda row: literal_eval(row))
# df.reset_index(drop=True, inplace=True)

# apple, samsung, canon 1:1:1
apple = extract_sample(data, 'Apple', 500, ratio = 1)
samsung = extract_sample(data, 'Samsung', 500, ratio = 1)
canon = extract_sample(data, 'Canon', 500, ratio = 1)
df = pd.concat([apple, samsung, canon], axis=0)
df.reset_index(drop=True, inplace=True)

In [23]:
# Apple 리뷰에서만
apple = extract_sample(data, 'Apple', 2500, ratio = 0)
df = apple.copy()
df.reset_index(drop=True, inplace=True)
# df.to_csv(work_path + "apple_data.csv", index=False)

In [8]:
df = pd.read_csv(work_path + 'apple_data.csv' )

In [9]:
print(df[df.overall >= 4].shape)
print(df[df.overall <= 2].shape)

(3307, 12)
(1324, 12)


In [11]:
# prepare
sentence_list, sentiment_label, sentence_senti_label, \
pos_neg_sentence_indices, pos_neg_sentiment_label, numSentence = prepare(df)

documents, sentence_list_again, bigram, documents_label\
= bigram_and_sentence(sentence_senti_label, sentence_list, numSentence, max_vocab=5000, threshold = 5, min_count = 20)

/home/hs-ubuntu/.local/lib/python3.5/site-packages/gensim/models/phrases.py:248: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [12]:
len(documents)

2507014

In [29]:
asum = ASUM(pos_seed, neg_seed, numTopics=10, alpha=0.01, beta=0.001, gamma=1, numSentiments=2)
asum._initialize_(sentence_list_again, pos_neg_sentence_indices, pos_neg_sentiment_label, sentiment_label)

In [44]:
t = [86, 80, 72]

[86, 80, 72]

In [61]:
with open(work_path + "BagOfSentences.txt", 'w') as f:
    for doc in range(asum.numDocs):
        f.write(str(len(asum.doc_sent_word_dict[doc])))
        f.write("\n")
        for sent in asum.doc_sent_word_dict[doc]:
            for word in sent:
                f.write(str(word) + ' ')
            f.write("\n")

In [57]:
asum.doc_sent_word_dict[0]

[[278, 878, 817, 445, 410],
 [34, 138, 122, 30, 310, 86, 636, 1896, 229, 6, 1415, 122]]

In [56]:
with open(work_path + "BagOfSentences.txt", 'w') as f:
    for sent in sent_index:
        f.write(str(len(sent)))
        f.write("\n")
        for word in sent:
            f.write(str(word) + ' ')
        f.write("\n")

In [33]:
from collections import OrderedDict
dic = OrderedDict(sorted(asum.word2idx.items(), key=lambda t: t[1]))
word_list = []
for key in dic.keys():
    word_list.append(key)
with open(work_path + "WordList.txt", 'w') as f:
    for word in word_list:
        f.writelines(word + "\n")

#### 그냥 ASUM

In [21]:
%%time
asum_path = "E:/dataset/MasterThesis/Models/ASUM_test1_"
asum = ASUM(pos_seed, neg_seed, numTopics=10, alpha=0.01, beta=0.001, gamma=1, numSentiments=2)
asum._initialize_(sentence_list_again, pos_neg_sentence_indices, pos_neg_sentiment_label, sentiment_label)
asum.run(sentence_list_again, save_path=asum_path, print_iter=2, save_iter = 50, maxIters= 100)

Starting iteration 2 of 100
0.5615
Starting iteration 4 of 100
0.567
Starting iteration 6 of 100
0.5765
Starting iteration 8 of 100
0.5885
Starting iteration 10 of 100
0.6
Starting iteration 12 of 100
0.614
Starting iteration 14 of 100
0.633
Starting iteration 16 of 100
0.6365
Starting iteration 18 of 100
0.6305
Starting iteration 20 of 100
0.6175
Starting iteration 22 of 100
0.625
Starting iteration 24 of 100
0.621
Starting iteration 26 of 100
0.611
Starting iteration 28 of 100
0.6215
Starting iteration 30 of 100
0.615
Starting iteration 32 of 100
0.611
Starting iteration 34 of 100
0.608
Starting iteration 36 of 100
0.609
Starting iteration 38 of 100
0.602
Starting iteration 40 of 100
0.6005
Starting iteration 42 of 100
0.601
Starting iteration 44 of 100
0.597
Starting iteration 46 of 100
0.593
Starting iteration 48 of 100
0.5995
Starting iteration 50 of 100
0.5975
Starting save model
Starting iteration 52 of 100
0.6
Starting iteration 54 of 100
0.598
Starting iteration 56 of 100
0.59

In [43]:
%%time
asum_path = "E:/dataset/MasterThesis/Models/ASUM_test2_"
asum = ASUM(pos_seed, neg_seed, numTopics=10, alpha=0.01, beta=0.001, gamma=1, numSentiments=2)
asum._initialize_(sentence_list_again, pos_neg_sentence_indices, pos_neg_sentiment_label, sentiment_label)
asum.run(sentence_list_again, save_path=asum_path, print_iter=2, save_iter = 50, maxIters= 100)

Starting iteration 2 of 100
0.499
Starting iteration 4 of 100
0.481
Starting iteration 6 of 100
0.479
Starting iteration 8 of 100
0.494
Starting iteration 10 of 100
0.493
Starting iteration 12 of 100
0.505666666667
Starting iteration 14 of 100
0.483333333333
Starting iteration 16 of 100
0.480666666667
Starting iteration 18 of 100
0.478666666667
Starting iteration 20 of 100
0.462666666667
Starting iteration 22 of 100
0.464
Starting iteration 24 of 100
0.460666666667
Starting iteration 26 of 100
0.451666666667
Starting iteration 28 of 100
0.456666666667
Starting iteration 30 of 100
0.454
Starting iteration 32 of 100
0.452
Starting iteration 34 of 100
0.445
Starting iteration 36 of 100
0.453
Starting iteration 38 of 100
0.447333333333
Starting iteration 40 of 100
0.442
Starting iteration 42 of 100
0.455333333333
Starting iteration 44 of 100
0.451333333333
Starting iteration 46 of 100
0.453333333333
Starting iteration 48 of 100
0.453333333333
Starting iteration 50 of 100
0.448
Starting sav

In [44]:
%%time
asum_path = "E:/dataset/MasterThesis/Models/ASUM_test3_"
asum = ASUM(pos_seed, neg_seed, numTopics=10, alpha=0.01, beta=0.001, gamma=1, numSentiments=2)
asum._initialize_(sentence_list_again, pos_neg_sentence_indices, pos_neg_sentiment_label, sentiment_label)
asum.run(sentence_list_again, save_path=asum_path, print_iter=2, save_iter = 50, maxIters= 100)

Starting iteration 2 of 100
0.520666666667
Starting iteration 4 of 100
0.522
Starting iteration 6 of 100
0.523333333333
Starting iteration 8 of 100
0.501
Starting iteration 10 of 100
0.496
Starting iteration 12 of 100
0.506333333333
Starting iteration 14 of 100
0.506333333333
Starting iteration 16 of 100
0.514666666667
Starting iteration 18 of 100
0.518
Starting iteration 20 of 100
0.519
Starting iteration 22 of 100
0.522666666667
Starting iteration 24 of 100
0.526333333333
Starting iteration 26 of 100
0.530333333333
Starting iteration 28 of 100
0.526
Starting iteration 30 of 100
0.527666666667
Starting iteration 32 of 100
0.526
Starting iteration 34 of 100
0.520666666667
Starting iteration 36 of 100
0.525666666667
Starting iteration 38 of 100
0.519333333333
Starting iteration 40 of 100
0.514
Starting iteration 42 of 100
0.515333333333
Starting iteration 44 of 100
0.518333333333
Starting iteration 46 of 100
0.518
Starting iteration 48 of 100
0.508
Starting iteration 50 of 100
0.5123333

In [16]:
len(pos_neg_sentence_indices)

6000

In [17]:
len(pos_neg_sentiment_label)

5553

In [19]:
pos_neg_sentiment_label[:10]

[0, 0, 1, 0, 0, 1, 0, 1, 0, 0]